In [1]:
import praw
import time
import pandas as pd
import csv
from praw.models import Submission
import os
from tenacity import retry, stop_after_attempt, wait_exponential
from tqdm import tqdm 

In [2]:
# Define necessary names
SUBREDDIT_NAME = "UBC"       
MAX_POSTS = 10000                       
BATCH_SIZE = 100                       
CHECKPOINT_FILE = "checkpoint.txt"      
POSTS_CSV = "r_ubc_posts.csv"                 
COMMENTS_CSV = "r_ubc_comments.csv"           
REQUEST_DELAY = 1

In [3]:
# Load reddit api throgh praw
reddit = praw.Reddit(
    client_id="qg6qmpLE5NHZ98F71GBiiA",
    client_secret="5flQrSnLQjGwIoLQIrV_YFluxej9GQ",
    user_agent="script:my_script:v1.0 (by /u/NathanPalaiologos)"
)



In [4]:
# Set csv file

pd.DataFrame(columns=[
    "post_id", "title", "author", "content",
    "score", "num_comments", "created_utc", "url"
]).to_csv(POSTS_CSV, index=False)


pd.DataFrame(columns=[
    "post_id", "comment_id", "author", "body", 
    "score", "created_utc"
]).to_csv(COMMENTS_CSV, index=False)

In [5]:
posts_file = open("r_ubc_posts.csv", "w", newline="", encoding="utf-8")
comments_file = open("r_ubc_comments.csv", "w", newline="", encoding="utf-8")

In [6]:
posts_writer = csv.writer(posts_file)
comments_writer = csv.writer(comments_file)
posts_writer.writerow(["id", "title", "author", "selftext", "score", "num_comments", "created_utc", "url"])
comments_writer.writerow(["post_id", "comment_id", "author", "body", "score", "created_utc"])


50

In [7]:
subreddit_name = "UBC" 
subreddit = reddit.subreddit(subreddit_name)

In [8]:
# check end points
# try:
#     with open(CHECKPOINT_FILE, "r") as f:
#         processed_ids = set(f.read().splitlines())
# except FileNotFoundError:
#     processed_ids = set()

# # initialize data cache
# posts_buffer = []
# comments_buffer = []

# @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
# def process_submission(submission):
#     """处理单个帖子（含重试机制）"""
#     posts_buffer.append({
#         "post_id": submission.id,
#         "title": submission.title,
#         "author": str(submission.author),
#         "selftext": submission.selftext,
#         "score": submission.score,
#         "num_comments": submission.num_comments,
#         "created_utc": submission.created_utc,
#         "url": submission.url
#     })

#     # comment limit
#     submission.comments.replace_more(limit=10)  
#     for comment in submission.comments.list():
#         comments_buffer.append({
#             "post_id": submission.id,
#             "comment_id": comment.id,
#             "author": str(comment.author),
#             "body": comment.body,
#             "score": comment.score,
#             "created_utc": comment.created_utc
#         })

# def save_batch():
#     """保存当前批次数据"""
#     if posts_buffer:
#         pd.DataFrame(posts_buffer).to_csv(POSTS_CSV, mode="a", header=False, index=False)
#         posts_buffer.clear()
#     if comments_buffer:
#         pd.DataFrame(comments_buffer).to_csv(COMMENTS_CSV, mode="a", header=False, index=False)
#         comments_buffer.clear()


# try:
#     subreddit = reddit.subreddit(SUBREDDIT_NAME)
#     total_collected = 0

#     with tqdm(total=MAX_POSTS, desc="Crawling Progress") as pbar:
#         for submission in subreddit.new(limit=None):
#             if total_collected >= MAX_POSTS:
#                 break

#             # skip scraped posts
#             if submission.id in processed_ids:
#                 continue

#             try:
#                 process_submission(submission)
#                 total_collected += 1
#                 pbar.update(1)

#                 # update endpoints
#                 with open(CHECKPOINT_FILE, "a") as f:
#                     f.write(f"{submission.id}\n")

#                 if total_collected % BATCH_SIZE == 0:
#                     save_batch()

#                 # add delay
#                 time.sleep(REQUEST_DELAY)

#             except Exception as e:
#                 print(f"\n处理帖子 {submission.id} 时出错: {str(e)}")
#                 continue

#     # save data
#     save_batch()

# finally:
#     # save all
#     save_batch()
#     print("\n爬取已终止，最后批次数据已保存")

In [9]:
# Process comments
def process_comment(comment, post_id, csv_writer):
    csv_writer.writerow([
        post_id,
        comment.id,
        comment.author,
        comment.body,
        comment.score,
        comment.created_utc
    ])
    for reply in comment.replies:
        process_comment(reply, post_id, csv_writer)

# collect data
try:
    for submission in subreddit.new(limit=10000):
        print(f"Posts: {submission.title}")
        posts_writer.writerow([
            submission.id,
            submission.title,
            submission.author,
            submission.selftext,
            submission.score,
            submission.num_comments,
            submission.created_utc,
            submission.url
        ])

        # comments collection
        submission.comments.replace_more(limit=10)
        for comment in submission.comments.list():
            process_comment(comment, submission.id, comments_writer)

        time.sleep(1)  # Set delay to avoid blocking
finally:
    posts_file.close()
    comments_file.close()

Posts: Does the UBC Lost and Found keep Apple Pencils?
Posts: Econ 317 friends
Posts: 2nd YEAR ELECMAXXING (ELEC 221)
Posts: Classes require discussions, team projects, and presentations, but I have severe anxiety
Posts: go global results!!
Posts: No longer snowing over the weekend
Posts: sweatpants for the cold
Posts: 24hrs parking on W 16th Ave
Posts: Vacation During Co-op Term
Posts: the hoods on the oversized zip up sweaters are goated
Posts: study spots with a view
Posts: Do you think there will be a Snow Day next week?
Posts: Where the Mature studentd at?
Posts: Theater and Opera
Posts: ESADE Barcelona Go Global
Posts: ADVICE NEEDED SCHOLARS!
Posts: awesome & fun club opportunity for creatives
Posts: How to gift something to my crush on valentine day
Posts: Are they reviewing my files already or is it just ready for evaluation?
Posts: Anyone want roses and other flowers?
Posts: “Fundraising” in IKB??
Posts: Is there any discord group for next year Faculty of Applied Sciences ? An